# 🧪 QEPC Sandbox v2

A safe playground to experiment with the QEPC NBA engine:

1. Environment & imports  
2. (Optional) System diagnostics  
3. Load schedule & select games  
4. Build team strengths  
5. (Optional) Apply injury overrides  
6. Compute λ (expected points)  
7. Run Poisson simulations  
8. View and explore results  
9. (Optional) Interactive widgets


## 1. Environment & Project Setup


In [1]:
import sys
from pathlib import Path

# Try to detect the QEPC project root
cwd = Path.cwd()
candidate_roots = [cwd, cwd.parent, cwd.parent.parent]

project_root = None
for cand in candidate_roots:
    if (cand / "qepc").is_dir() and (cand / "qepc_autoload.py").exists():
        project_root = cand
        break

if project_root is None:
    project_root = cwd

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("📁 QEPC project root set to:", project_root)

try:
    from notebook_context import *
    print("✅ notebook_context imported.")
except ImportError:
    print("ℹ️ notebook_context not found; continuing without it.")

try:
    import qepc_autoload as qa
    print("✅ qepc_autoload imported as qa.")
except Exception as e:
    print("❌ Error importing qepc_autoload:", e)


📁 QEPC project root set to: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project
[QEPC Paths] Project Root set: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project
[QEPC] Autoload complete.
[QEPC] Root Shim Restored. Forwarding to qepc.autoload...
✅ notebook_context imported.
✅ qepc_autoload imported as qa.


## 2. System Diagnostics (Optional)


In [2]:
# === QEPC Sandbox: System Diagnostics (Optional) ===

from qepc.utils.diagnostics import run_system_check

diagnostic_report = run_system_check()
diagnostic_report  # Shows root/files/modules info as a dict


🚀 QEPC SYSTEM DIAGNOSTICS INITIALIZED...

✅ Project Root: OK – Resolved to /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project

🔍 Checking required files...
✅ Canonical Schedule: OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Games.csv
✅ Raw Player Stats: OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/PlayerStatistics.csv
✅ Raw Team Stats: OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/TeamStatistics.csv
✅ Autoload Context: OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/qepc_autoload.py
✅ Restore Guide (Notebook): OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/RESTORE_GUIDE.ipynb
✅ Restore Guide (Markdown): OK – /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/notebooks/RESTORE_GUIDE.md


Check,Status,Details
Canonical Schedule,OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Games.csv
Raw Player Stats,OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/PlayerStatistics.csv
Raw Team Stats,OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/TeamStatistics.csv
Autoload Context,OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/qepc_autoload.py
Restore Guide (Notebook),OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/RESTORE_GUIDE.ipynb
Restore Guide (Markdown),OK,/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/notebooks/RESTORE_GUIDE.md



📊 Checking data schemas (where files exist)...
✅ Schema: data/Games.csv: OK – All expected columns present.
✅ Schema: data/Team_Stats.csv: OK – All expected columns present.
✅ Schema: data/raw/PlayerStatistics.csv: OK – All expected columns present.
✅ Schema: data/raw/TeamStatistics.csv: OK – All expected columns present.


Check,Status,Details
data/Games.csv,OK,All expected columns present.
data/Team_Stats.csv,OK,All expected columns present.
data/raw/PlayerStatistics.csv,OK,All expected columns present.
data/raw/TeamStatistics.csv,OK,All expected columns present.



🧪 Checking key module imports...
✅ Module: qepc.autoload.paths: OK – Loaded
✅ Module: qepc.core.lambda_engine: OK – Loaded
✅ Module: qepc.core.simulator: OK – Loaded
✅ Module: qepc.sports.nba.sim: OK – Loaded
✅ Module: qepc.sports.nba.strengths_v2: OK – Loaded
✅ Module: qepc.sports.nba.player_data: OK – Loaded
✅ Module: qepc.sports.nba.opponent_data: OK – Loaded
✅ Module: qepc.utils.backup: OK – Loaded
✅ Module: qepc.backtest.backtest_engine: OK – Loaded


Check,Status,Details
qepc.autoload.paths,OK,Loaded successfully
qepc.core.lambda_engine,OK,Loaded successfully
qepc.core.simulator,OK,Loaded successfully
qepc.sports.nba.sim,OK,Loaded successfully
qepc.sports.nba.strengths_v2,OK,Loaded successfully
qepc.sports.nba.player_data,OK,Loaded successfully
qepc.sports.nba.opponent_data,OK,Loaded successfully
qepc.utils.backup,OK,Loaded successfully
qepc.backtest.backtest_engine,OK,Loaded successfully



✨ DIAGNOSTICS COMPLETE.


{'project_root': '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project',
 'files': [('Canonical Schedule',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Games.csv'),
  ('Raw Player Stats',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/PlayerStatistics.csv'),
  ('Raw Team Stats',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/TeamStatistics.csv'),
  ('Autoload Context',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/qepc_autoload.py'),
  ('Restore Guide (Notebook)',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/RESTORE_GUIDE.ipynb'),
  ('Restore Guide (Markdown)',
   'OK',
   '/home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/notebooks/RESTORE_GUIDE.md')],
 'schemas': [('data/Games.csv', 'OK', 'All expected columns present.'),
  ('data/Team_Stats.csv', 'OK', 'All expected columns present.'),
  ('data/raw/PlayerStatistics.csv', 'OK', 'All expect

## 3. Load NBA Schedule


In [3]:
# === QEPC Sandbox: Load NBA Schedule ===

import qepc_autoload as qa

schedule = qa.load_nba_schedule()
print("Number of games in schedule:", len(schedule))
schedule.head()


[QEPC NBA Sim] Successfully loaded and parsed 771 games from original format.
Number of games in schedule: 771


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00
4,10/22/2025,7:00 PM,Miami Heat,Orlando Magic,Kia Center,Regular Season,2025-10-22 19:00:00


## 4. Select Games to Model


In [4]:
# === QEPC Sandbox: Select Games to Model ===

# Option A: first 4 games (e.g., opening night)
games_to_model = schedule.head(4).copy()

print("Using these games:")
games_to_model


Using these games:


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00


## 5. Build Advanced Team Strengths


In [5]:
# === QEPC Sandbox: Build Advanced Team Strengths ===

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

advanced_strengths = calculate_advanced_strengths()
print("Raw advanced_strengths shape:", advanced_strengths.shape)

# Collapse to one row per team
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

print("Unique teams in advanced strengths:", len(advanced_team_strengths))
advanced_team_strengths.head()


[QEPC Strength V2] Starting Advanced Calculation (Cutoff: Now)...
[QEPC PlayerData] Successfully loaded 1635462 rows from PlayerStatistics.csv.
[QEPC Opponent Processor] Loading raw team data for Weighted DRtg...
[QEPC Opponent Processor] Calculated Weighted DRtg for 30 teams.
[QEPC Strength V2] Calculated Time-Travel Strengths for 30 teams.
Raw advanced_strengths shape: (30, 5)
Unique teams in advanced strengths: 30


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424


## 6. Injury Overrides (Optional)


In [6]:
# === QEPC Sandbox: Injury Overrides (Optional) ===

import pandas as pd

inj_path = project_root / "data" / "Injury_Overrides.csv"

if inj_path.exists():
    injuries = pd.read_csv(inj_path)
    print("✅ Loaded injury overrides from:", inj_path)
    display(injuries)

    # Collapse to team-level impact (multiply multiple entries per team)
    team_injury_impact = (
        injuries
        .groupby("Team", as_index=False)["Impact"]
        .prod()
        .rename(columns={"Impact": "InjuryImpact"})
    )

    print("Team-level InjuryImpact:")
    display(team_injury_impact)

    # Merge into strengths
    inj_adjusted = advanced_team_strengths.merge(
        team_injury_impact,
        on="Team",
        how="left"
    )

    # Teams without overrides get impact = 1.0 (no change)
    inj_adjusted["InjuryImpact"] = inj_adjusted["InjuryImpact"].fillna(1.0)

    # Apply to ORtg (you can also apply to Pace if you want)
    inj_adjusted["ORtg_inj"] = inj_adjusted["ORtg"] * inj_adjusted["InjuryImpact"]

    print("Injury-adjusted team strengths (first 5):")
    display(inj_adjusted.head())

    # This is the table we feed into the λ engine
    team_strengths_for_lambda = inj_adjusted.copy()
    team_strengths_for_lambda["ORtg"] = team_strengths_for_lambda["ORtg_inj"]

    # Optional clean-up of helper columns
    for col in ["ORtg_inj", "InjuryImpact"]:
        if col in team_strengths_for_lambda.columns:
            team_strengths_for_lambda.drop(columns=[col], inplace=True)

else:
    print("ℹ️ No Injury_Overrides.csv found at", inj_path)
    print("Using unadjusted advanced strengths.")
    team_strengths_for_lambda = advanced_team_strengths.copy()


✅ Loaded injury overrides from: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Injury_Overrides.csv


,Team,PlayerName,Status,Impact,Note
0,Indiana Pacers,Tyrese Haliburton,out,0.85,Out for season


Team-level InjuryImpact:


,Team,InjuryImpact
0,Indiana Pacers,0.85


Injury-adjusted team strengths (first 5):


,Team,ORtg,DRtg,Pace,Volatility,InjuryImpact,ORtg_inj
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725,1.0,122.0
1,Boston Celtics,122.0,107.711196,68.080000,12.410859,1.0,122.0
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107,1.0,122.0
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697,1.0,122.0
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424,1.0,122.0


## 7. Compute Lambda (Expected Points)


In [7]:
# === QEPC Sandbox: Compute Lambda (Expected Points) ===

from qepc.core.lambda_engine import compute_lambda

lambda_df = compute_lambda(games_to_model, team_strengths_for_lambda)

print("Lambda dataframe columns:")
print(lambda_df.columns.tolist())

# Adjust these column names if needed based on the printout above
display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
    ]
)


[QEPC Lambda] Computed lambda & volatility for 4 games.
Lambda dataframe columns:
['Date', 'Time', 'Away Team', 'Home Team', 'Venue', 'Notes', 'gameDate', 'lambda_home', 'lambda_away', 'vol_home', 'vol_away']


,Away Team,Home Team,lambda_away,lambda_home,vol_away,vol_home
0,Houston Rockets,Oklahoma City Thunder,87.431249,96.878531,32.325436,14.033294
1,Golden State Warriors,Los Angeles Lakers,100.679142,101.130359,10.256705,13.236314
2,Brooklyn Nets,Charlotte Hornets,107.385905,112.664166,9.097107,13.022697
3,Cleveland Cavaliers,New York Knicks,103.931072,107.670313,12.175616,13.069948


## 8. Run Simulation & View Results


In [8]:
# === QEPC Sandbox: Run QEPC Simulation ===

from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Simulation result columns:")
print(sim_results.columns.tolist())
sim_results.head()


[QEPC Simulator] Running 20000 trials for 4 games (Chaos Engine Active)...
[QEPC Simulator] Simulation complete.
Simulation result columns:
['Date', 'Time', 'Away Team', 'Home Team', 'Venue', 'Notes', 'gameDate', 'lambda_home', 'lambda_away', 'vol_home', 'vol_away', 'Home_Win_Prob', 'Away_Win_Prob', 'Tie_Prob', 'Expected_Score_Total', 'Expected_Spread', 'Sim_Home_Score', 'Sim_Away_Score']


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate,lambda_home,lambda_away,vol_home,vol_away,Home_Win_Prob,Away_Win_Prob,Tie_Prob,Expected_Score_Total,Expected_Spread,Sim_Home_Score,Sim_Away_Score
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00,96.878531,87.431249,14.033294,32.325436,0.65845,0.32495,0.01660,184.35855,9.39635,96.87745,87.48110
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00,101.130359,100.679142,13.236314,10.256705,0.50210,0.47455,0.02335,201.84615,0.58785,101.21700,100.62915
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00,112.664166,107.385905,13.022697,9.097107,0.61550,0.36110,0.02340,219.92240,5.46610,112.69425,107.22815
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00,107.670313,103.931072,13.069948,12.175616,0.56995,0.40975,0.02030,211.83445,3.48545,107.65995,104.17450


## 9. Interactive QEPC Controls (Widgets)


In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from datetime import datetime, timedelta

from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation


# --- Build a filtered "upcoming games" view from schedule ---

def ensure_game_datetime(df: pd.DataFrame) -> pd.Series:
    """
    Ensure we have a datetime column for each game.
    Prefer 'gameDate' if it exists; otherwise parse from Date + Time.
    """
    if "gameDate" in df.columns:
        return pd.to_datetime(df["gameDate"])
    # Fallback: parse Date + Time strings
    return pd.to_datetime(df["Date"] + " " + df["Time"])


def format_game_option(row):
    """Turn a schedule row into a nice label for the dropdown."""
    return f"{row['Date']} {row['Time']} – {row['Away Team']} @ {row['Home Team']}"


# 1) Compute game datetimes
game_dt = ensure_game_datetime(schedule)

# 2) Define "today" and the 3-day window
today = pd.Timestamp.today().normalize()
horizon = today + pd.Timedelta(days=3)

mask = (game_dt >= today) & (game_dt < horizon)
upcoming = schedule.loc[mask].copy()

# If no games in the next 3 days, fall back to next 20 games on the schedule
if upcoming.empty:
    print("ℹ️ No games found in the next 3 days. Showing next 20 games instead.")
    upcoming = schedule.sort_values(game_dt.name).head(20).copy()

# Build (label, original_index) options
game_options = []
for idx, row in upcoming.iterrows():
    label = format_game_option(row)
    game_options.append((label, idx))

game_dropdown = widgets.Dropdown(
    options=game_options,
    description='Game:',
    layout=widgets.Layout(width='95%')
)

# Keep the trials slider
num_trials_slider = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step=1000,
    description='Trials:',
    continuous_update=False
)

run_button = widgets.Button(
    description="Run QEPC Sim",
    button_style='success',
    tooltip='Run QEPC for the selected game'
)

output = widgets.Output()


def on_run_clicked(b):
    with output:
        clear_output()

        # 1) Get the selected game (index refers back to full schedule)
        game_idx = game_dropdown.value
        game_row = schedule.loc[[game_idx]].copy()  # DataFrame with a single row

        print("Running QEPC for:")
        display(game_row[["Date", "Time", "Away Team", "Home Team", "Venue", "Notes"]])

        # 2) Compute lambda for this single game
        lambda_df = compute_lambda(game_row, team_strengths_for_lambda)

        print("\nLambda (expected points) for this game:")
        display(
            lambda_df[
                ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
            ]
        )

        # 3) Run simulation
        trials = num_trials_slider.value
        print(f"\nRunning simulation with {trials} trials...\n")
        sim_results = run_qepc_simulation(lambda_df, num_trials=trials)

        # 4) Show a compact summary (adapt names if your sim uses slightly different ones)
        cols = [
            "Away Team", "Home Team",
            "Home_Win_Prob", "Away_Win_Prob",
            "Expected_Score_Total", "Expected_Spread",
            "Sim_Home_Score", "Sim_Away_Score"
        ]
        cols = [c for c in cols if c in sim_results.columns]

        print("QEPC summary for this matchup:")
        display(sim_results[cols])


run_button.on_click(on_run_clicked)

controls = widgets.VBox([
    widgets.HTML("<h3>🎛 QEPC Interactive Controls</h3>"),
    game_dropdown,
    num_trials_slider,
    run_button
])

display(widgets.VBox([controls, output]))

## 10. Data-Driven Injury Impact from PlayerStatistics

This section uses `data/raw/PlayerStatistics.csv` to estimate how much of each
team's offense an injured player was responsible for, and uses that to set
the `Impact` multiplier instead of guessing by hand.


In [10]:
import pandas as pd

# --- 10.1 Load PlayerStatistics from the QEPC data folder ---

ps_path = project_root / "data" / "raw" / "PlayerStatistics.csv"
print("Loading PlayerStatistics from:", ps_path)

player_stats = pd.read_csv(
    ps_path,
    low_memory=False,
    parse_dates=["gameDate"]
)

player_stats.head()

Loading PlayerStatistics from: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/raw/PlayerStatistics.csv


,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Jamal,Murray,1627750,22500248,2025-11-17T21:00:00Z,Denver,Nuggets,Chicago,Bulls,NaN,...,0.455,5.0,5.0,1.0,11.0,0.0,11.0,3.0,2.0,-1.0
1,Bruce,Brown,1628971,22500248,2025-11-17T21:00:00Z,Denver,Nuggets,Chicago,Bulls,NaN,...,0.000,0.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,-17.0
2,Jevon,Carter,1628975,22500248,2025-11-17T21:00:00Z,Chicago,Bulls,Denver,Nuggets,NaN,...,0.500,0.0,0.0,0.0,3.0,1.0,4.0,2.0,1.0,20.0
3,Kevin,Huerter,1628989,22500248,2025-11-17T21:00:00Z,Chicago,Bulls,Denver,Nuggets,NaN,...,0.444,2.0,2.0,1.0,2.0,0.0,2.0,0.0,1.0,-21.0
4,Jalen,Pickett,1629618,22500248,2025-11-17T21:00:00Z,Denver,Nuggets,Chicago,Bulls,NaN,...,1.000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,9.0


In [13]:
# --- 10.2 Filter to recent games (e.g. 2024-25 & 2025-26) ---

import pandas as pd

# Cutoff date (no timezone)
cutoff_date = pd.Timestamp("2024-10-01")

# 1) Convert gameDate to datetime, force into UTC, then drop timezone to make it tz-naive
player_stats["gameDate"] = pd.to_datetime(
    player_stats["gameDate"],
    errors="coerce",
    utc=True          # make everything timezone-aware in UTC
).dt.tz_localize(None)  # then strip timezone info → plain datetime

# 2) Now comparison is tz-naive vs tz-naive → valid
ps_recent = player_stats[player_stats["gameDate"] >= cutoff_date].copy()

print("Recent rows:", len(ps_recent))
ps_recent.head()


Recent rows: 8024


,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Jamal,Murray,1627750,22500248,2025-11-17 21:00:00,Denver,Nuggets,Chicago,Bulls,NaN,...,0.455,5.0,5.0,1.0,11.0,0.0,11.0,3.0,2.0,-1.0
1,Bruce,Brown,1628971,22500248,2025-11-17 21:00:00,Denver,Nuggets,Chicago,Bulls,NaN,...,0.000,0.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,-17.0
2,Jevon,Carter,1628975,22500248,2025-11-17 21:00:00,Chicago,Bulls,Denver,Nuggets,NaN,...,0.500,0.0,0.0,0.0,3.0,1.0,4.0,2.0,1.0,20.0
3,Kevin,Huerter,1628989,22500248,2025-11-17 21:00:00,Chicago,Bulls,Denver,Nuggets,NaN,...,0.444,2.0,2.0,1.0,2.0,0.0,2.0,0.0,1.0,-21.0
4,Jalen,Pickett,1629618,22500248,2025-11-17 21:00:00,Denver,Nuggets,Chicago,Bulls,NaN,...,1.000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,9.0
